<a href="https://colab.research.google.com/github/nasibehmohammadi/Thesis/blob/main/1lstm_d8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_true = K.cast(y_true, dtype=tf.float32)

        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())

        cross_entropy = -y_true * K.log(y_pred)

        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        return K.sum(loss, axis=-1)

    return focal_loss_fixed

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

file_path = "/content/d8.csv"
df = pd.read_csv(file_path)

print("🔍 بررسی ۵ سطر اول دیتاست:")
print(df.head())

feature_columns = ["A","B","C","D","E"]
target_column = "F"

X_data = df[feature_columns].values
y_data = df[target_column].values

num_features = X_data.shape[1]

time_steps = 30

X_sequences = []
y_sequences = []

for i in range(len(X_data) - time_steps):
    X_sequences.append(X_data[i : i + time_steps])
    y_sequences.append(y_data[i + time_steps])

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train = X_train.reshape(-1, num_features)
X_test = X_test.reshape(-1, num_features)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(-1, time_steps, num_features)
X_test = X_test.reshape(-1, time_steps, num_features)

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(time_steps, num_features)),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss=focal_loss(gamma=2., alpha=0.25), metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ دقت مدل روی داده‌های تست: {accuracy * 100:.2f}%")

test_sample = X_test[0].reshape(1, time_steps, num_features)
prediction = model.predict(test_sample)

if prediction > 0.5:
    print("🚨 ورود مشکوک! نیاز به احراز هویت بیشتر.")
else:
    print("✅ ورود ایمن است.")

🔍 بررسی ۵ سطر اول دیتاست:
   A  B  C  D  E  F
0  4  3  8  8  9  1
1  0  3  3  0  1  0
2  5  0  4  3  0  0
3  0  7  4  3  4  0
4  6  9  5  4  8  0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 16s 41ms/step - accuracy: 0.6879 - loss: 0.0039 - val_accuracy: 0.6996 - val_loss: 2.1629e-07
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 38ms/step - accuracy: 0.7055 - loss: 2.1823e-06 - val_accuracy: 0.6996 - val_loss: 5.7854e-08
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.6942 - loss: 9.2248e-07 - val_accuracy: 0.6996 - val_loss: 2.7910e-08
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 18s 33ms/step - accuracy: 0.7120 - loss: 6.2541e-07 - val_accuracy: 0.6996 - val_loss: 1.6304e-08
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.7042 - loss: 4.6857e-07 - val_accuracy: 0.6996 - val_loss: 9.6024e-09
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.7090 - loss: 2.8582e-07 - val_accuracy: 0.6996 - val_loss: 6.7636e-09
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.7063 - loss: 2.3750e-07 - val_accuracy: 0.6996 - val_loss: 4.7442e-09
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
from sklearn.metrics import f1_score

# 1️⃣ پیش‌بینی بر روی مجموعه تست
y_pred = model.predict(X_test)

# 2️⃣ تبدیل احتمالات به برچسب‌های 0 و 1 (اگر مقدار احتمال > 0.5 باشد، کلاس 1 است، در غیر این صورت 0)
y_pred_classes = (y_pred > 0.5).astype(int)

# 3️⃣ محاسبه F1-Score
f1 = f1_score(y_test, y_pred_classes)

print(f"🎯 F1-Score مدل: {f1:.4f}")


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
🎯 F1-Score مدل: 0.8233
